In [111]:
import json
import glob
from pprint import pprint
import random

In [120]:
s = 100

l = [[[0],[0]],[[1],[1]],[[2],[2]],[[3],[3]]]

a = len(l)

z = int(a * (50/100))
print(z)

r = random.sample(l, k=z)
print(r)

2
[[[1], [1]], [[0], [0]]]


In [108]:
# trn data
val_dir = '/disk/project/koreanvalidation/input/val/'
val_files = glob.glob(val_dir+'*')

# trn_save dir
trn_save_fname = '/disk/project/corpus/val.conll'

In [63]:
print(val_files)

['/disk/project/koreanvalidation/input/val/NWRW1800000032-0017.json', '/disk/project/koreanvalidation/input/val/NWRW1800000029-0156.json', '/disk/project/koreanvalidation/input/val/NWRW1800000021-0026.json', '/disk/project/koreanvalidation/input/val/NWRW1800000041-0329.json', '/disk/project/koreanvalidation/input/val/NWRW1800000026-0039.json', '/disk/project/koreanvalidation/input/val/NWRW1800000052-0284.json', '/disk/project/koreanvalidation/input/val/NWRW1800000033-0369.json', '/disk/project/koreanvalidation/input/val/NWRW1800000022-0081.json', '/disk/project/koreanvalidation/input/val/NWRW1800000044-0172.json', '/disk/project/koreanvalidation/input/val/NWRW1800000037-0146.json', '/disk/project/koreanvalidation/input/val/NWRW1800000044-0246.json', '/disk/project/koreanvalidation/input/val/NWRW1800000041-0332.json', '/disk/project/koreanvalidation/input/val/NWRW1800000041-0055.json', '/disk/project/koreanvalidation/input/val/NWRW1800000042-0142.json', '/disk/project/koreanvalidation/i

In [14]:
def get_word_id(item, word):
    word_id = False
    e = item['end']
    for i in range(len(word)):
        w = word[i]
        if w['begin'] <= e <= w['end']:
            word_id = i
            break

    return word_id

In [24]:
def words2tokens(words):
    tokens = []
    for i in words:
        tokens.append(i['form'])
    return tokens

In [54]:
def convert_for_sent(srls, words):
    result = []
    tokens = words2tokens(words)
    for srl in srls:
        if srl:
            pred_id = get_word_id(srl['predicate'], words)
            pred = srl['predicate']['lemma'] + '.' + str(srl['predicate']['sense_id'])            
            
            preds = ['_' for i in range(len(tokens))]
            senses = ['_' for i in range(len(tokens))]
            preds[pred_id] = 'PRED'
            senses[pred_id] = pred
            
            args = ['O' for i in range(len(tokens))]
            
            for arg in srl['argument']:
                arg_id = get_word_id(arg, words)
                label = arg['label'].replace('-', '_')
                args[arg_id] = label
                
            sent = []
            sent.append(tokens)
            sent.append(preds)
            sent.append(senses)
            sent.append(args)
            
            result.append(sent)
            
    return result

In [91]:
def converter(files, save_fname):
    conll = []
    
    doc_list = []
    for fname in files:        
        docid = fname.split('/')[-1].split('.')[0]
        doc_list.append(docid)
        
        with open(fname, 'r') as f:
            d = json.load(f)
            
        for doc in d['document']:
            for sent in doc['sentence']:
                words = sent['word']
                srl = sent['SRL']
                srl_conll = convert_for_sent(srl, words)
                
                conll += srl_conll
                
    doc_ids = ','.join(doc_list)                
    with open(save_fname, 'w') as f:
        f.write('#doc_ids: '+doc_ids+'\n')
        for sent in conll:
            tokens, preds, senses, args = sent[0], sent[1], sent[2], sent[3]            
            n = 0
            for i in range(len(tokens)):
                line = str(n) + '\t' + tokens[i] + '\t' + preds[i] + '\t' + senses[i] + '\t' + args[i] + '\n'
                n +=1
                f.write(line)
            f.write('\n')
        
    print('saved to:', save_fname)
    print('saved srls:', len(conll))

In [109]:
# convert train data
converter(val_files, trn_save_fname)

saved to: /disk/project/corpus/val.conll
saved srls: 4488


In [83]:
n = 7192

x = int(n / 10) +1
print(x)

720


In [110]:
# test data
const_dir = '/disk/project/corpus/dataArchive/7_srl/const/converted/written_0217/'
const_files = glob.glob(const_dir+'*')

def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]
        
devided_const_files = list(chunks(const_files, int(len(const_files)/10)+1))


n = 1
for files in devided_const_files:
    save_fname = '/disk/project/corpus/test_'+str(n)+'.conll'
    converter(files, save_fname)
    n +=1
    
ns = [1,2,3,4,5,6,7,8,9,10]
with open('./srl_splits.txt','w') as f1:
    for n in ns:
        fname = '/disk/project/corpus/test_'+str(n)+'.conll'
        with open(fname, 'r') as f2:
            x = f2.readlines()
            
        docid = x[0]
        line = '\t' + docid + '\n'
        f1.write(line)

saved to: /disk/project/corpus/test_1.conll
saved srls: 42665
saved to: /disk/project/corpus/test_2.conll
saved srls: 43655
saved to: /disk/project/corpus/test_3.conll
saved srls: 43439
saved to: /disk/project/corpus/test_4.conll
saved srls: 43094
saved to: /disk/project/corpus/test_5.conll
saved srls: 42562
saved to: /disk/project/corpus/test_6.conll
saved srls: 45155
saved to: /disk/project/corpus/test_7.conll
saved srls: 43780
saved to: /disk/project/corpus/test_8.conll
saved srls: 43507
saved to: /disk/project/corpus/test_9.conll
saved srls: 42879
saved to: /disk/project/corpus/test_10.conll
saved srls: 42043


In [102]:
with open('./data/tag2idx.json') as f:
    tag2idx = json.load(f)

def error_checker(files):
    conll = []
    
    doc_list = []
    for fname in files:        
        docid = fname.split('/')[-1].split('.')[0]
        
        with open(fname, 'r') as f:
            d = json.load(f)
            
        for doc in d['document']:
            for sent in doc['sentence']:
                sent_id = sent['id']
                words = sent['word']
                srl = sent['SRL']
                srl_conll = convert_for_sent(srl, words)
                
                for i in srl_conll:
                    args = i[-1]
                    
                    for a in args:
                        if a not in tag2idx:
                            print(sent_id, a)

In [103]:
# error checker
const_dir = '/disk/project/corpus/dataArchive/7_srl/const/converted/written_0217/'
const_files = glob.glob(const_dir+'*')

error_checker(const_files)

NWRW1800000038-0234.32 >>>
NWRW1800000030-0145.4 ARGM_LOC(장소)
NWRW1800000041-0097.5 ARR2
